In [7]:
import pandas as pd
import scipy
import sklearn
import numpy as np
import pickle
import setuptools
import wheel
import matplotlib.pyplot as plt
import docplex.mp.model as cpx
import random

import pulp as plp
from pulp import LpProblem, LpInteger, LpVariable, LpConstraint, LpConstraintGE, LpConstraintLE, LpConstraintEQ, LpMaximize, lpSum

In [8]:
# Parameters

RANK = range(1,11)
SCORE = [6,3,2,1,0,0,0,0,0,0]

# ROOMS = {
#     'RoomA': [7,16,4,8],
#     'RoomB': [9,10,14,1,3],
#     'RoomC': [12,17,13,2,18],
#     'RoomD': [6,11,5,15],
#     'RoomE': [6,11,5,15],
#     'RoomF': [6,11,5,15]
# }

ROOMS = {
    'a': [1, 3, 14],
    'b': [2, 4, 11],
    'c': [5, 12, 13],
    'd': [6, 9, 10],
    'e': [7, 16, 17],
    'f': [8, 15, 18]
}

In [9]:
def gen_room(room_dict):
    """ Append the word "Pair" to the pair number
    """
    
    output_dict = {}
    for key in room_dict.keys():
        templist = [''.join(['pair ', str(room)]) for room in room_dict[key]]
        output_dict[key] = templist
    return output_dict

def sum_room(old, room_dict):
    """ Sum scores of all rooms of each candidate
    """
    data = old.copy()
    #for room in room_dict.keys():
    #    data[room] = data[room_dict[room]].sum(axis = 1)
    data['RoomA'] = old['pair 4'] + old['pair 7'] + old['pair 8'] + old['pair 16']
    data['RoomB'] = old['pair 1'] + old['pair 3'] + old['pair 9'] + old['pair 10'] + old['pair 14']
    data['RoomC'] = old['pair 12'] + old['pair 17'] + old['pair 13'] + old['pair 2'] + old['pair 18']
    data['RoomD'] = old['pair 5'] + old['pair 6'] + old['pair 11'] + old['pair 15']
    return data

In [10]:
df = pd.read_csv('./res/champ_6_pair_rank.csv').fillna(0)

# * rename columns
# cols = ['id'] 
# cols.extend(df.columns[1:])
# df.columns = cols

In [11]:
# Transform rank to score
transform_dict = {k: v for k, v in zip(RANK, SCORE)}
df_score = df.replace(transform_dict)
df_score.head()

,id,pair_1,pair_2,pair_3,pair_4,pair_5,pair_6,pair_7,pair_8,pair_9,pair_10,pair_11,pair_12,pair_13,pair_14,pair_15,pair_16,pair_17,pair_18
0,6330002721,2.0,3.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,13.0,6.0,6.0,0.0,0.0,0.0
1,6330013621,2.0,0.0,6.0,0.0,0.0,13.0,6.0,0.0,0.0,6.0,6.0,0.0,0.0,3.0,0.0,0.0,0.0,6.0
2,6330022221,2.0,6.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,13.0,3.0,0.0,6.0,0.0,0.0
3,6330034821,0.0,6.0,6.0,0.0,0.0,2.0,13.0,0.0,0.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0
4,6330038321,0.0,0.0,6.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,6.0,2.0,6.0,6.0,13.0,0.0,0.0


In [12]:
new = sum_room(df_score, gen_room(ROOMS))
room_score = new[['ID', 'RoomA', 'RoomB', 'RoomC', 'RoomD']]
#room_score_unranked = new[['ID', 'RoomA', 'RoomB', 'RoomC', 'RoomD']]
room_score.head()

KeyError: 'pair 4'

# Define Variables for Optimization

In [16]:
I = range(0, df.shape[0]) # candidate index
J = range(1, len(ROOMS.keys()) + 1) # room index

# maximum number of candidates in each room
max_num = {1: 24, 2: 32, 3: 32, 4: 24}
min_num = {1: 23, 2: 25, 3: 25, 4: 20}

## Constraints

In [17]:
# Note: i == candidate's index, j == room's index
rank = {(i, j): room_score.iloc[i, j] for i in I for j in J}
lower_b = {(i, j): 0 for i in I for j in J}
upper_b = {(i, j): 1 for i in I for j in J}
a = upper_b.copy()

# everyone can join only one room
one_c = {(i): 1 for i in I}

# satisfaction score (need to be updated due to our level)
# (direct / grey plus / grey)
sat_c = {(i): 4 for i in I[:41]}
sat_c.update({(i): 3 for i in I[41:41 + 29]})
sat_c.update({(i): 2 for i in I[41 + 29:41 + 29 + 27]})
sat_c.update({(i): 1 for i in I[41 + 29 + 27:]})

In [18]:
#rank

## Model Setup

In [19]:
model = LpProblem(name = "score_optimizer")

# Assign lower bound and upper bound to DVs
x_vars = {
    (i, j): LpVariable(cat = LpInteger, lowBound = lower_b[i, j],
                       upBound = upper_b[i, j],
                       name = 'x_{0}_{1}'.format(i, j))
    for i in I for j in J
}

# Room minimum and maximum number constraint
cts = {j: model.addConstraint(LpConstraint(e = lpSum(a[i, j] * x_vars[i, j]
                                                     for i in I), sense = LpConstraintLE, rhs = max_num[j],
                                           name = 'max_number_room_{0}'.format(j))) for j in J}
cts = {j: model.addConstraint(LpConstraint(e = lpSum(a[i, j] * x_vars[i, j]
                                                     for i in I), sense = LpConstraintGE, rhs = min_num[j],
                                           name = 'min_number_room_{0}'.format(j))) for j in J}

# A candidate must join only one room
cts = {i: model.addConstraint(LpConstraint(e = lpSum(a[i, j] * x_vars[i, j]
                                                     for j in J), sense = LpConstraintEQ, rhs = one_c[i],
                                           name = 'one_room_candidate_{0}'.format(i))) for i in I}

# Minimum Score
cts = {i: model.addConstraint(LpConstraint(e = lpSum(rank[i, j] * x_vars[i, j] 
                                                     for j in J), sense = LpConstraintGE, rhs = sat_c[i],
                                           name = 'min_satisfaction_{0}'.format(i))) for i in I}

In [20]:
# Set objective
model.sense = LpMaximize
obj = lpSum(x_vars[i, j] * rank[i, j] for i in I for j in J)
model.setObjective(obj)

# Solve!
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/pasin/Codes/Python/ChAMPEng/env/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/cg/6xrmgcbj71nfsp9hqcxq2fzh0000gn/T/a26502c64a984ae592936aea1293c05b-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/cg/6xrmgcbj71nfsp9hqcxq2fzh0000gn/T/a26502c64a984ae592936aea1293c05b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 227 COLUMNS
At line 2982 RHS
At line 3205 BOUNDS
At line 3634 ENDATA
Problem MODEL has 222 rows, 428 columns and 1591 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 766 - 0.00 seconds
Cgl0003I 128 fixed, 0 tightened bounds, 0 strengthened rows, 36 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 1 substitutions
Cgl0004I processed model has 37 rows, 108 columns (108 integer (105 of which bin

1

In [21]:
plp.LpStatus[model.status]

'Optimal'

In [22]:
# Total Satisfaction Score
summary = []
for i in I:
    for j in J:
        summary.append(plp.value(x_vars[i, j] * rank[i, j]))

## Convert the result to a DataFrame

In [23]:
opt_df = pd.DataFrame.from_dict(x_vars, orient = 'index',
                                columns = ['variable_object'])
opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, names = ['candidate_id', 'room'])
opt_df.reset_index(inplace = True)

opt_df['solution_value'] = opt_df['variable_object'].apply(lambda item: item.varValue)

final_table = opt_df.pivot(index = 'candidate_id', columns = 'room', values = 'solution_value')

In [24]:
final_table.head()

room,1,2,3,4
candidate_id,,,,
0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0


In [17]:
temp_final = final_table.reset_index(drop = True)
temp_final['ID'] = room_score['ID']

In [18]:
temp_final.head()

room,1,2,3,4,ID
0,0.0,0.0,1.0,0.0,6230124521
1,0.0,1.0,0.0,0.0,6030274221
2,1.0,0.0,0.0,0.0,6031779021
3,0.0,1.0,0.0,0.0,6030650521
4,1.0,0.0,0.0,0.0,6131791621


In [25]:
temp_final.to_excel('final_flex_no_ranked.xlsx', index = False)

In [20]:
sum(summary)

829.0

In [21]:
room_score.head()

,ID,RoomA,RoomB,RoomC,RoomD
0,6230124521,0.0,2.0,9.0,1.0
1,6030274221,0.0,6.0,3.0,3.0
2,6031779021,6.0,4.0,2.0,0.0
3,6030650521,0.0,11.0,0.0,1.0
4,6131791621,6.0,4.0,2.0,0.0


In [22]:
df_score.head()

,ID,pair 1,pair 2,pair 3,pair 4,pair 5,pair 6,pair 7,pair 8,pair 9,pair 10,pair 11,pair 12,pair 13,pair 14,pair 15,pair 16,pair 17,pair 18
0,6230124521,2.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
1,6030274221,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0
2,6031779021,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6030650521,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,6131791621,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [23]:
temp_final.sum(axis = 0).astype(int)

room
1               23
2               29
3               32
4               24
ID    658274929168
dtype: int64